<a href="https://colab.research.google.com/github/Toshi-M56/PIDcontrol/blob/main/%E5%B7%A5%E5%AD%A6%E5%AE%9F%E9%A8%93PID%E5%88%B6%E5%BE%A1_%E3%82%B9%E3%83%86%E3%83%83%E3%83%97%E5%BF%9C%E7%AD%94%E5%BC%8F%E3%81%AE%E6%8C%99%E5%8B%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install control
!pip install japanize_matplotlib
import numpy as np
from control import matlab
from matplotlib import pyplot as plt
import japanize_matplotlib
plt.style.use('bmh')
from matplotlib import animation, rc
from IPython.display import HTML

In [ ]:
def function(t, Kp, Kd, show_D=True):
    D = 12.25*Kd**2 - 7*Kp

    if abs(D) < 1e-10:
        if show_D:print("D=" + str(D) + " :D = 0")
        term1 = np.ones(t.shape)
        term21 = -np.ones(t.shape)
        term22 = np.exp(-3.5*Kd*t) 
        term2 = term21 * term22
        term31 = -np.ones(t.shape) * (-3.5*Kd*t)
        term32 = np.exp(-3.5*Kd*t) 
        term3 =  term31 * term32
    elif D > 0:
        if show_D:print("D=" + str(D) + " :D > 0")
        beta = np.sqrt(12.25*Kd**2 - 7*Kp)
        # print(Kp, Kd, beta)
        p1 = -3.5*Kd + beta
        p2 = -3.5*Kd - beta
        # return 1 - (1 / (2*beta))* (p1*np.exp(p1*t) - p2*np.exp(p2*t))
        term1 = np.ones(t.shape)
        term21 = -(1 / (2*beta)) *term1
        term22 = p1*np.exp(p1*t) 
        term2 = term21 * term22
        term31 =  (1 / (2*beta)) *term1
        term32 =  p2*np.exp(p2*t)
        term3 =  term31 * term32
    elif D < 0:
        if show_D:print("D=" + str(D) + " :D < 0")
        omega = np.sqrt(-12.25*Kd**2 + 7*Kp)
        # return 1 - np.exp(-3.5*Kd*t) * (np.cos(omega*t) - (3.5*Kd/omega)*np.sin(omega*t))
        term1 = np.ones(t.shape)
        term21 = -np.cos(omega*t)
        term22 = np.exp(-3.5*Kd*t)
        term2 = term21 * term22
        term31 =  (3.5*Kd/omega)*np.sin(omega*t)
        term32 =  np.exp(-3.5*Kd*t) 
        term3 = term31 * term32
    return term1, term2, term3, term21, term22, term31, term32

In [ ]:
def plot_terms(t, term1, term2, term3, term21, term22, term31, term32):
    fig = plt.figure(figsize=(6,10))
    ax = fig.add_subplot(311)
    ax.plot(t,term1, c="violet",ls="--", label="第1項")
    ax.plot(t,term2, c="green", ls="--", label="第2項")
    ax.plot(t,term3, c="orange", ls="--", label="第3項")
    ax.plot(t,term1+term2+term3, c="black", label="位置")
    ax.set_ylim([-0.5, 1.7])
    ax.legend()
    ax2 = fig.add_subplot(312)
    ax2.plot(t,term21, ls="--", label="term21")
    ax2.plot(t,term22, ls="--", label="term22")
    ax2.plot(t,term2, c="green", label="第2項")
    ax2.legend()
    # ax2.set_ylim([-1.5, 1.5])
    ax3 = fig.add_subplot(313)
    ax3.plot(t,term31, ls="--", label="term31")
    ax3.plot(t,term32, ls="--", label="term32")
    ax3.plot(t,term3, c="orange", label="第3項")
    ax3.legend()
    # ax3.set_ylim([-1.5, 1.5])
    plt.show()

##PD制御について復讐
PD制御
$C(s)=K_P + K_Ds$
をかけた閉ループ伝達関数は
$$\frac{C(s)P(s)}{1+C(s)P(s)}=\frac{7K_Ds + 7K_P}{s^2 + 7K_Ds + 7K_P}$$
であり，その極は分母多項式から，
$$s=-3.5Kd \pm \sqrt{12.25K_D^2 - 7K_P}$$
となりました．これは判別式$D=12.25K_D^2-7K_P$によって3つに場合わけされます．

1.   $D<0$ 極が極が虚数成分を持つ
$$s = -3.5K_D \pm j \sqrt{7K_P - 12.25K_D^2}$$
2.   $D=0$　極が重根
$$s = -3.5K_D $$
3.   $D>0$　極が異なる二つの実数
$$s = -3.5K_D \pm j \sqrt{12.25K_D^2 - 7K_P}$$

##PD制御のステップ応答について復讐
そして，ステップ応答は
$$\frac{7K_Ds + 7K_P}{s(s^2 + 7K_Ds + 7K_P)}=\frac{1}{s}-\frac{1}{s^2 + 7K_Ds + 7K_P}$$
となり，ステップ応答の時間領域での式も判別式によって3つに別れました．
1.   $D<0$ 極が極が虚数成分を持つ
$$x(t) = 1 - e^{-3.5K_Dt} \big(\cos{\omega t} - \frac{3.5K_D}{\omega}\sin{\omega t} \big) =  1 - e^{-3.5K_Dt} \cos{\omega t} + e^{-3.5K_Dt} \frac{3.5K_D}{\omega}\sin{\omega t} $$
$$\big( \omega = \sqrt{7K_P - 12.25K_D^2} \big)$$
2.   $D=0$　極が重根
$$x(t)=1-e^{-3.5K_D t}(1-3.5K_D t)$$
3.   $D>0$　極が異なる二つの実数
$$1-\frac{1}{2 \beta}(p_1 e^{p_1 t} - p_2 e^{p_2 t})$$

$$= 1 + \frac{3.5K_D - \sqrt{12.25K_D^2 - 7K_P}}{2 \sqrt{12.25K_D^2 -7K_P}} e^{-3.5K_D + \sqrt{12.25K_D^2 - 7K_P}} - \frac{3.5K_D + \sqrt{12.25K_D^2 - 7K_P}}{2 \sqrt{12.25K_D^2 -7K_P}} e^{-(3.5K_D + \sqrt{12.25K_D^2 - 7K_P})}$$



In [ ]:
Kp = 0.4
Kd = 0.1
t = np.linspace(0, 10, 1000)
term1, term2, term3, term21, term22, term31, term32 = function(t, Kp, Kd)
plot_terms(t, term1, term2, term3, term21, term22, term31, term32)

In [ ]:



fig = plt.figure()

ims = []
for a in range(100):
    Kp = 0.4
    Kd = round(0.01*a, 4)
    t = np.linspace(0, 10, 1000)
    term1, term2, term3, term21, term22, term31, term32 = function(t, Kp, Kd, show_D=False)
    line, = plt.plot(t, term1, ls="--", c="violet")
    line2, = plt.plot(t, term2, ls="--", c="green")
    line3, = plt.plot(t, term3, ls="--", c="orange")
    line4, = plt.plot(t, term1+term2+term3, c="black")
    plt.ylim([-1.2,2.3])
    text = plt.text(2,2.2, "Kp = " + str(Kp) + "  Kd = " + str(Kd), fontsize=20)
    ims.append([line, line2, line3, line4, text])

ani = animation.ArtistAnimation(fig, ims)
rc('animation', html='jshtml')
plt.close()
ani


In [ ]:
print(np.sqrt(7*Kp/12.25))
f = np.sqrt(7*Kp) / (2*np.pi)
print(1/f)